***
# Car prices
***

1. Répertorier l'ensemble des marques de véhicules
2. Répertorier l'ensemble des modèles de véhicules
3. Répertorier l'ensemble des prix des véhicules
***

In [1]:
# Create folders and getting path
import os
cwd = os.getcwd()

# Loop to create folders
folder_names = ['Dataframe', 'Output', 'Data']

folders = {}
for folder_name in folder_names:
    folders[folder_name] = os.path.join(cwd, folder_name)

    if not os.path.exists(os.path.join(cwd, folder_name)):
        os.makedirs(os.path.join(cwd, folder_name))
        print(f'Le dossier « {folder_name} » a été créé')

    else:
        print(f'Le dossier « {folder_name} » est existant')


# Création des variables de path

# Déterminer si os est win ou linux pour définir les path
if os.name == 'nt':
    slash = '\\'
elif os.name == 'posix':
    slash = '/'

path_prog =     cwd + slash
path_data =     folders['Data'] + slash
path_df =       folders['Dataframe'] + slash
path_output =   folders['Output'] + slash

# Mettre \\ pour éviter les erreurs
path_dict = [path_prog, path_data, path_df, path_output]
for path in path_dict:
    path = path.replace('\\','\\\\')


# Détermination de l'année
print()
print(path_prog)
print(path_data)
print(path_df)
print(path_output)

Le dossier « Dataframe » est existant
Le dossier « Output » est existant
Le dossier « Data » est existant

c:\Users\Charles_tour\Documents\GitHub\car_sales_predictions\
c:\Users\Charles_tour\Documents\GitHub\car_sales_predictions\Data\
c:\Users\Charles_tour\Documents\GitHub\car_sales_predictions\Dataframe\
c:\Users\Charles_tour\Documents\GitHub\car_sales_predictions\Output\


In [2]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

link_original        = 'https://www.guideautoweb.com'
link_constructeur   = 'https://www.guideautoweb.com/constructeurs/'
response_api = requests.get(link_constructeur)
print(response_api.status_code)     # 200 is ok

200


In [3]:
# Get the data from api
data = response_api.text
soup = BeautifulSoup(data, 'html.parser')

In [69]:
# Extract all marque and link
ul_element = soup.find('ul', id='brands-index-list')

data = {
    'marque' : [],
    'link_model' : []
}

# Extract information from <a> elements within the <ul> element
for a_element in ul_element.find_all('a'):
    a_text = a_element.get_text().strip()
    a_href = a_element.get('href')
    link_model = link_original + a_href

    data['marque'].append(a_text)
    data['link_model'].append(link_model)

df = pd.DataFrame(data)
df

,marque,link_model
0,Acura,https://www.guideautoweb.com/constructeurs/acura/
1,Alfa Romeo,https://www.guideautoweb.com/constructeurs/alf...
2,Allard,https://www.guideautoweb.com/constructeurs/all...
3,Aston Martin,https://www.guideautoweb.com/constructeurs/ast...
4,Audi,https://www.guideautoweb.com/constructeurs/audi/
...,...,...
73,Tesla,https://www.guideautoweb.com/constructeurs/tesla/
74,Toyota,https://www.guideautoweb.com/constructeurs/toy...
75,VinFast,https://www.guideautoweb.com/constructeurs/vin...
76,Volkswagen,https://www.guideautoweb.com/constructeurs/vol...


In [98]:
count = 0

data = {
    'model' : [],
    'link' : [],
    'production' : []
    }

for link in df['link_model']:
    if count >= 1:
        break  # Exit the loop after the second iteration

    model_api = requests.get(link)
    count += 1


    model_data = model_api.text
    model_soup = BeautifulSoup(model_data, 'html.parser')


    # Extract model en production
    section_production = model_soup.find_all('div', class_='s')


    extracted_text_list = []
    extracted_ref_list = []

    for section in section_production:
        try:
            a_elements = section.find_all('a', class_='e-a e-t')

            for a_element in a_elements:
                extracted_text_list.append(a_element.get_text())
                extracted_ref_list.append(a_element.get('href'))

        except AttributeError:
            continue

    for text, ref in zip(extracted_text_list, extracted_ref_list):
        data['model'].append(text)
        data['link'].append(link[:-1] + ref)
        data['production'].append(True)


    # Extracting model non en production
    section_production_autre = model_soup.find('ul', class_='eg eg-t1 eg-sz-s')
    if section_production_autre:
        for a_element in section_production_autre.find_all('a', class_='txt'):
            try:
                text = a_element.get_text()
                data['model'].append(text)
                data['link'].append(link[:-1] + a_element.get('href'))
                data['production'].append(False)
                
            except AttributeError:
                continue

    df_model = pd.DataFrame(data)
df_model    
    

,model,link,production
0,Acura Integra,https://www.guideautoweb.com/constructeurs/acu...,True
1,Acura MDX,https://www.guideautoweb.com/constructeurs/acu...,True
2,Acura RDX,https://www.guideautoweb.com/constructeurs/acu...,True
3,Acura TLX,https://www.guideautoweb.com/constructeurs/acu...,True
4,Acura CL,https://www.guideautoweb.com/constructeurs/acu...,False
5,Acura Concept,https://www.guideautoweb.com/constructeurs/acu...,False
6,Acura CSX,https://www.guideautoweb.com/constructeurs/acu...,False
7,Acura EL,https://www.guideautoweb.com/constructeurs/acu...,False
8,Acura ILX,https://www.guideautoweb.com/constructeurs/acu...,False
9,Acura NSX,https://www.guideautoweb.com/constructeurs/acu...,False


In [99]:
df_model.to_excel(path_output + 'df_model.xlsx')

In [87]:
print(df_model)

             model                                link  production
0    Acura Integra  /constructeurs/acura/integra/2023/        True
1        Acura MDX      /constructeurs/acura/mdx/2023/        True
2        Acura RDX      /constructeurs/acura/rdx/2023/        True
3        Acura TLX      /constructeurs/acura/tlx/2023/        True
4         Acura CL            /constructeurs/acura/cl/       False
..             ...                                 ...         ...
824      Volvo S40           /constructeurs/volvo/s40/       False
825      Volvo S80           /constructeurs/volvo/s80/       False
826      Volvo V50           /constructeurs/volvo/v50/       False
827      Volvo V70           /constructeurs/volvo/v70/       False
828     Volvo XC70          /constructeurs/volvo/xc70/       False

[829 rows x 3 columns]


In [88]:
model = df_model.iloc[0,1]
model

'/constructeurs/acura/integra/2023/'

In [78]:
# Extract information from Acura page
model_api = requests.get(model)
model_data = model_api.text
model_soup = BeautifulSoup(model_data, 'html.parser')

data = {
    'model' : [],
    'link' : [],
    'production' : []
    }


# Extract model en production
section_production = model_soup.find_all('div', class_='s')
print(section_production)
extracted_text_list = []
extracted_ref_list = []

for section in section_production:
    a_elements = section.find_all('a', class_='e-a e-t')

    for a_element in a_elements:
        extracted_text_list.append(a_element.get_text())
        extracted_ref_list.append(a_element.get('href'))

for text, ref in zip(extracted_text_list, extracted_ref_list):
    data['model'].append(text)
    data['link'].append(ref)
    data['production'].append(True)


# Extracting model non en production
section_production_autre = model_soup.find('ul', class_='eg eg-t1 eg-sz-s')

for a_element in section_production_autre.find_all('a', class_='txt'):
    text = a_element.get_text()
    ref = a_element.get('href')

    data['model'].append(text)
    data['link'].append(ref)
    data['production'].append(False)

df_test = pd.DataFrame(data)
df_test

[<div class="s ad ad-728x90" data-ad-network="dfp" data-position="top" id="ad-728x90-1"><div id="div-gpt-ad-c1dd29147a25c31c6117f2433a13170d-1"></div><script>googletag.cmd.push(function() { googletag.display("div-gpt-ad-c1dd29147a25c31c6117f2433a13170d-1"); });</script></div>, <div class="s" style="height: 700px">
<div class="s sticky ad ad-300x250" data-ad-network="dfp" data-position="top" id="ad-300x250-1"><div id="div-gpt-ad-c1dd29147a25c31c6117f2433a13170d-2"></div><script>googletag.cmd.push(function() { googletag.display("div-gpt-ad-c1dd29147a25c31c6117f2433a13170d-2"); });</script></div></div>, <div class="s sticky ad ad-300x250" data-ad-network="dfp" data-position="top" id="ad-300x250-1"><div id="div-gpt-ad-c1dd29147a25c31c6117f2433a13170d-2"></div><script>googletag.cmd.push(function() { googletag.display("div-gpt-ad-c1dd29147a25c31c6117f2433a13170d-2"); });</script></div>, <div class="s" id="search-box">
<div class="st"><span>Consulter</span> un véhicule</div>
<div style="margi

In [ ]:
# Extract list of all models
# Extract list of web links to the models

In [ ]:
# Extract list of 'model en production'
# Extract web link to those models

In [ ]:
# Extract price and fuel consomption for every model